In [2]:
!pip install --quiet --upgrade accelerate
!pip install git+https://github.com/huggingface/diffusers.git@main
!pip install controlnet_hinter==0.0.5
!pip install gradio
!pip install torch torchvision


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 55.1 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/diffusers.git (to revision main) to /tmp/pip-req-build-rspfxxi6
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-rspfxxi6
  Resolved https://github.com/huggingface/diffusers.git to commit 0d68ddf3275b20b0d12cfd3d0a9f002fecfe001c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.29.0.dev0-py3-none-any.whl size=2178796 sha256=8ec871d726c3d6bc0eb9cf27127c7333773c605a41037579b9c62a8cb02a97a3
  Stored in directory: /tmp/pip-ephem-wheel-cache-y3_mijh1/wheels/2f/e1/9c/c4e22972ee698b020d29ed7cc914d898247d01e302f25df1a4
Successfully built diffusers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
IMG_PATH = "/content/drive/MyDrive/It-Jim/It-Jim_logo.png"
SAVE_FOLDER_PATH = "/content/drive/MyDrive/It-Jim/generated_logs/"

In [5]:
import controlnet_hinter
import matplotlib.pyplot as plt
from PIL import Image
import os
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
import gradio as gr

# Define the controlnet mapping
CONTROLNET_MAPPING = {
    "hough": {
        "model_id": "lllyasviel/sd-controlnet-mlsd",
        "hinter": controlnet_hinter.hint_hough,
    }
}

# Define model path for Juggernaut
base_model_path = "digiplay/Juggernaut_final"

# Initialize pipeline for Hough control type to run on GPU
controlnet_type = "hough"
controlnet = ControlNetModel.from_pretrained(CONTROLNET_MAPPING[controlnet_type]["model_id"], torch_dtype=torch.float16).to("cuda")
pipe = StableDiffusionControlNetPipeline.from_pretrained(base_model_path,
                                                         controlnet=controlnet,
                                                         torch_dtype=torch.float16,
                                                         ).to("cuda")


/usr/local/lib/python3.10/dist-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.w

config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.45G [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/593 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

...

tokenizer/tokenizer_config.json:   0%|          | 0.00/737 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [6]:
# Function to generate image using the Juggernaut model with Hough control type
def generate_image_with_hough_control(prompt, negative_prompt, no_of_steps, guidance_scale, controlnet_conditioning_scale):

    logo_image = Image.open(IMG_PATH)
    control_image = CONTROLNET_MAPPING[controlnet_type]["hinter"](logo_image)

    # Run the inference
    my_images = pipe(
        prompt=prompt,
        width=512,  # Reduced image size to save memory
        height=256,  # Reduced image size to save memory
        negative_prompt=negative_prompt,
        image=control_image,
        controlnet_conditioning_scale=controlnet_conditioning_scale,
        num_inference_steps=no_of_steps,
        guidance_scale=guidance_scale,
    )

    # Get the first image from the image generations object
    first_image = my_images.images[0]

    # Save the first generated image to local directory
    save_dir = SAVE_FOLDER_PATH
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f"{controlnet_type}_{prompt[:15]}.png")
    first_image.save(save_path)
    print(f"Image saved to: {save_path}")

    return first_image

# Define Gradio interface
interface = gr.Interface(
    fn=generate_image_with_hough_control,
    inputs=[
        # gr.Image(type="filepath", label="Logo Image"),
        gr.Textbox(lines=2, placeholder="Enter your prompt here...", label="Prompt"),
        gr.Textbox(lines=2, placeholder="Enter negative prompt here...", label="Negative Prompt"),
        gr.Slider(minimum=1, maximum=50, value=20, step=1, label="Number of Steps"),
        gr.Slider(minimum=1.0, maximum=20.0, value=7.0, step=0.1, label="Guidance Scale"),
        gr.Slider(minimum=0.1, maximum=5.0, value=1.0, step=0.1, label="ControlNet Conditioning Scale"),
    ],
    outputs=gr.Image(type="pil", label="Generated Image"),
    title="It-Jim Logo Generator with Hough Control",
    description="Generate a new It-Jim logo using the Juggernaut model with Hough control type."
)


interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://4d790d68667f87fcb7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
